<a href="https://colab.research.google.com/github/RohithOfRivia/Analysing-Conusmer-Behaviour-MRP/blob/main/28/03/23-3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [1]:
!pip install rdkit
!pip install -U mlxtend
!pip install Boruta

import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

from boruta import BorutaPy

import joblib
import sys
sys.modules['sklearn.externals.joblib'] = joblib
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from imblearn.over_sampling import BorderlineSMOTE

from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors
from rdkit.Chem import MACCSkeys

from sklearn.cluster import KMeans

from sklearn.svm import OneClassSVM
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier, LocalOutlierFactor
from sklearn.ensemble import (GradientBoostingClassifier, 
                              HistGradientBoostingClassifier, IsolationForest)
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, RepeatedStratifiedKFold, cross_val_score, cross_val_predict
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler, LabelEncoder, OrdinalEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder

from sklearn.decomposition import PCA
from sklearn.feature_selection import VarianceThreshold
from sklearn.utils import resample

from sklearn.metrics import f1_score, recall_score

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold

from xgboost import XGBClassifier
import time
import os
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 13.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.1 MB/s eta 0:00:00
  Attempting uninstall: mlxtend
    Found existing installation: mlxtend 0.14.0
    Uninstalling mlxtend-0.14.0:
      Successfully uninstalled mlxtend-0.14.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 KB 5.8 MB/s eta 0:00:00


# Preprocessing and pipeline

In [2]:
train_data_url = "https://raw.githubusercontent.com/RohithOfRivia/SMILES-Toxicity-Prediction/main/Data/train_II.csv"
test_data_url = "https://raw.githubusercontent.com/RohithOfRivia/SMILES-Toxicity-Prediction/main/Data/test_II.csv"

df = pd.read_csv(train_data_url)

In [3]:
#transforming each compound into their canonical SMILES format. Optional.
def canonicalSmiles(smile):
    try:
        return Chem.MolToSmiles(Chem.MolFromSmiles(smile))
    except:
        return(Chem.MolToSmiles(Chem.MolFromSmiles("[Na+].[Na+].F[Si--](F)(F)(F)(F)F")))

In [4]:
#Read data and split up the given features
class FileReadTransform(BaseEstimator, TransformerMixin):
    
    def fit(self, X, y=None):
        return self
    
    #training and test data are slightly different, hence passing optional test param
    def transform(self, X, test=False):
        
        try:
          # if test == False:
            X['SMILES'] = X['Id'].apply(lambda x: x.split(';')[0])
            X['assay'] = X['Id'].apply(lambda x: x.split(';')[1])
        
        except KeyError:
            X['SMILES'] = X['x'].apply(lambda x: x.split(';')[0])
            X['assay'] = X['x'].apply(lambda x: x.split(';')[1])
          
        print("FileReadTransform done")
        
        #correct smiles for this compound found through https://www.molport.com/shop/index
        #X["SMILES"] = X["SMILES"].replace({"F[Si-2](F)(F)(F)(F)F.[Na+].[Na+]":"[Na+].[Na+].F[Si--](F)(F)(F)(F)F"})
        
        #Deleting invalid compound from the data
        X = X.loc[X.SMILES != "F[Si-2](F)(F)(F)(F)F.[Na+].[Na+]"]
        return X
    
    
class CanonicalGenerator(BaseEstimator, TransformerMixin):
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X['SMILES'] = X['SMILES'].apply(canonicalSmiles)
        print("CanonicalGenerator done")
        return X


class Scaler(BaseEstimator, TransformerMixin):
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        temp_df = X[X.columns[-208:]]
        ss = StandardScaler()
        temp_df = pd.DataFrame(ss.fit_transform(temp_df), columns = temp_df.columns[-208:])

        return pd.concat([X[X.columns[:4]], temp_df], axis=1)

    
class FingerprintGenerator(BaseEstimator, TransformerMixin):
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
          #tracks each unique compound and its fingerprints
          tracker = []
          fps = []
          assays = []
          unique = len(X['SMILES'].unique())
          counter = 0

          for index, columns in X[["SMILES", "assay"]].iterrows():

              #skip if already in tracker
              if columns[0] in tracker:
                  continue

              #append each unique compound and thier respective fingerprints
              else:
                  tracker.append(columns[0])
                  assays.append(columns[1])

                  mol = Chem.MolFromSmiles(columns[0])
                  fp = AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=256)
                  fps.append(fp.ToList())

                  counter += 1

                  # print(f"compound {counter}/{unique}...

          #Combining all compounds, assays and fingerprints into one dataframe 
          cols = a = ["x" + str(i) for i in range (1, 257)]
          smiles_df = pd.DataFrame(columns=['SMILES'], data=tracker)
          fingerprints = pd.DataFrame(columns=cols, data=fps)

          df = pd.concat([smiles_df, fingerprints], axis=1)

          print("FingerprintGenerator done")
          return pd.merge(X, df, on='SMILES') 


class FingerprintGeneratorM(BaseEstimator, TransformerMixin):
    

    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
      
          #tracks each unique compound and its fingerprints
          tracker = []
          fps = []
          assays = []
          unique = len(X['SMILES'].unique())
          counter = 0

          for index, columns in X[["SMILES", "assay"]].iterrows():

              #skip if already in tracker
              if columns[0] in tracker:
                  continue

              #append each unique compound and thier respective fingerprints
              else:
                  

                  tracker.append(columns[0])
                  assays.append(columns[1])

                  mol = Chem.MolFromSmiles(columns[0])
                  fp = MACCSkeys.GenMACCSKeys(mol)
                  fps.append(fp.ToList())

                  counter += 1

                  # print(f"compound {counter}/{unique}...

          #Combining all compounds, assays and fingerprints into one dataframe 
          cols = a = ["x" + str(i) for i in range (1, 168)]
          smiles_df = pd.DataFrame(columns=['SMILES'], data=tracker)
          fingerprints = pd.DataFrame(columns=cols, data=fps)

          df = pd.concat([smiles_df, fingerprints], axis=1)

          print("FingerprintGenerator done")
          return pd.merge(X, df, on='SMILES') 


#Feature reduction with variance threshold 
class VarianceThresh(BaseEstimator, TransformerMixin):

    def fit(self, X, y=None):
        return self
    
    def transform(self, X, thresh=.8):
      
      #Looks to columns to determine whether X is training or testing data 
      cols = X.columns
      if 'x' in cols:
        temp_df = X.drop(columns=["x", "assay", "SMILES"])
        cols = ["x", "assay", "SMILES"]
      else:
        temp_df = X.drop(columns=["Id", "Expected","assay", "SMILES"])
        cols = ["Id", "Expected","assay", "SMILES"]

      #Selecting features based on the variance threshold
      selector = VarianceThreshold(threshold=(thresh * (1 - thresh))) 
      selector.fit(temp_df)

      #This line transforms the data while keeping the column names 
      temp_df = temp_df.loc[:, selector.get_support()]

      #Attaching the ids, assays, smiles etc. that is still required for model
      return pd.concat([X[cols], temp_df], axis=1) , selector


#Scale descriptors 
class Scaler(BaseEstimator, TransformerMixin):

    def fit(self, X, y=None):
      return self
    
    def transform(self, X):
      scaler = StandardScaler()

      if 'Id' in X.columns:
        temp_df = X.drop(columns=["Id", "Expected", "assay", "SMILES"])
        cols = ["Id", "Expected","assay", "SMILES"]

        X_scaled = pd.DataFrame(scaler.fit_transform(temp_df), columns=temp_df.columns)
        X = pd.concat([X[cols].reset_index(drop=True), X_scaled], axis=1)
          
        return X

      else:
        temp_df = X.drop(columns=["x", "assay", "SMILES"])
        cols = ["x", "assay", "SMILES"]

        X_scaled = pd.DataFrame(scaler.fit_transform(temp_df), columns=temp_df.columns)
        X = pd.concat([X[cols].reset_index(drop=True), X_scaled], axis=1)

        return X


#Scale descriptors 
class Encode(BaseEstimator, TransformerMixin):

    encoder = LabelEncoder()
    
    def fit(self, X, y=None):
      return self
    
    def transform(self, X):
      enc = LabelEncoder()

      if 'Id' in X.columns:
        temp_df = X.drop(columns=["Id", "Expected", "assay", "SMILES"])
        cols = ["Id", "Expected","assay", "SMILES"]

        X_scaled = pd.DataFrame(scaler.fit_transform(temp_df), columns=temp_df.columns)
        X = pd.concat([X[cols].reset_index(drop=True), X_scaled], axis=1)
          
        return X

      else:
        temp_df = X.drop(columns=["x", "assay", "SMILES"])
        cols = ["x", "assay", "SMILES"]

        X_scaled = pd.DataFrame(scaler.fit_transform(temp_df), columns=temp_df.columns)
        X = pd.concat([X[cols].reset_index(drop=True), X_scaled], axis=1)

        return X


# Generating descriptors

In [5]:
class DescriptorGenerator(BaseEstimator, TransformerMixin):

  def fit(self, X, y=None):
        return self
    
  def transform(self, X):
    #Initializing descriptor calculator
    calc = MoleculeDescriptors.MolecularDescriptorCalculator([x[0] for x in Descriptors._descList])
    desc_names = calc.GetDescriptorNames()

    #Tracking each unique compound and generating descriptors 
    tracker = []
    descriptors = []
    for compound in X['SMILES']:

      if compound in tracker:
        continue

      else:
        tracker.append(compound)

        mol = Chem.MolFromSmiles(compound)
        current_descriptors = calc.CalcDescriptors(mol)
        descriptors.append(current_descriptors)

    # Combining X, SMILES, and generated descriptors 
    df = pd.DataFrame(descriptors,columns=desc_names)
    temp_df = pd.DataFrame(tracker, columns=["SMILES"])
    df = pd.concat([df, temp_df], axis=1)

    print("DescriptorGenerator done")
    return pd.merge(X, df, on='SMILES')

In [6]:
feature_generation_pipeline = Pipeline(steps=[
    ('read', FileReadTransform()),
     ('canon', CanonicalGenerator()),
     ('fpr', FingerprintGenerator()),
     ('desc', DescriptorGenerator())
     ])

feature_selector_pipeline = Pipeline(steps=[
    ('vtr', VarianceThresh()),
     ])
df_processed = feature_generation_pipeline.fit_transform(df)
test_processed = feature_generation_pipeline.fit_transform(pd.read_csv(test_data_url))

FileReadTransform done
CanonicalGenerator done
FingerprintGenerator done
DescriptorGenerator done
FileReadTransform done
CanonicalGenerator done
FingerprintGenerator done
DescriptorGenerator done


# Feature Selection

In [7]:
descriptors = pd.concat([df_processed[['Id','SMILES','assay', 'Expected']], df_processed[df_processed.columns[-208:]]], axis=1)
descriptors

,Id,SMILES,assay,Expected,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,C1=CC(=CC=C1C(C2=CC=C(C=C2)O)C(Cl)(Cl)Cl)O;1644,Oc1ccc(C(c2ccc(O)cc2)C(Cl)(Cl)Cl)cc1,1644,2,9.316200,-1.533785,9.316200,0.150485,0.794714,317.599,...,0,0,0,0,0,0,0,0,0,0
1,C1=CC(=CC=C1C(C2=CC=C(C=C2)O)C(Cl)(Cl)Cl)O;1630,Oc1ccc(C(c2ccc(O)cc2)C(Cl)(Cl)Cl)cc1,1630,2,9.316200,-1.533785,9.316200,0.150485,0.794714,317.599,...,0,0,0,0,0,0,0,0,0,0
2,C1=CC(=CC=C1C(C2=CC=C(C=C2)O)C(Cl)(Cl)Cl)O;29,Oc1ccc(C(c2ccc(O)cc2)C(Cl)(Cl)Cl)cc1,29,2,9.316200,-1.533785,9.316200,0.150485,0.794714,317.599,...,0,0,0,0,0,0,0,0,0,0
3,C1=CC(=CC=C1C(C2=CC=C(C=C2)O)C(Cl)(Cl)Cl)O;1618,Oc1ccc(C(c2ccc(O)cc2)C(Cl)(Cl)Cl)cc1,1618,2,9.316200,-1.533785,9.316200,0.150485,0.794714,317.599,...,0,0,0,0,0,0,0,0,0,0
4,C1=CC(=CC=C1C(C2=CC=C(C=C2)O)C(Cl)(Cl)Cl)O;1638,Oc1ccc(C(c2ccc(O)cc2)C(Cl)(Cl)Cl)cc1,1638,2,9.316200,-1.533785,9.316200,0.150485,0.794714,317.599,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75372,C1=CC=C(C=C1)NC(=S)N;1852,NC(=S)Nc1ccccc1,1852,1,5.244798,0.297407,5.244798,0.297407,0.596319,152.222,...,0,0,0,0,0,0,0,0,0,0
75373,CN1CN(CN(C1)C)C;2,CN1CN(C)CN(C)C1,2,2,2.281250,1.087639,2.281250,1.087639,0.443158,129.207,...,0,0,0,0,0,0,0,0,0,0
75374,CCCCC1CCC(=O)O1;1852,CCCCC1CCC(=O)O1,1852,1,10.594398,-0.012868,10.594398,0.012868,0.562255,142.198,...,0,0,0,0,0,0,0,0,0,0
75375,CCOC(=O)CCC1=CC=CC=C1;2,CCOC(=O)CCc1ccccc1,2,2,10.994151,-0.119118,10.994151,0.119118,0.660429,178.231,...,0,0,0,0,0,0,0,0,0,0


In [8]:
descriptors.isna().sum().sum()
boruta_features2 = ['HeavyAtomMolWt',
 'MaxPartialCharge',
 'MinAbsPartialCharge',
 'BertzCT',
 'Chi1',
 'Chi1n',
 'Chi2v',
 'Chi3n',
 'Chi3v',
 'Chi4v',
 'LabuteASA',
 'PEOE_VSA3',
 'SMR_VSA6',
 'SMR_VSA7',
 'EState_VSA8',
 'VSA_EState2',
 'VSA_EState4',
 'HeavyAtomCount',
 'NumAromaticCarbocycles',
 'MolLogP',
 'MolMR',
 'fr_Ar_OH',
 'fr_COO',
 'fr_COO2',
 'fr_C_O',
 'fr_C_O_noCOO',
 'fr_amide',
 'fr_benzene',
 'fr_phenol',
 'fr_phenol_noOrthoHbond',
 'fr_sulfonamd',
 'fr_thiazole',
 'fr_thiophene',
 'fr_urea',
 'assay']

In [9]:
descriptors
descriptors2 = descriptors.drop(columns=['BCUT2D_MWHI', 'BCUT2D_MWLOW', 'BCUT2D_CHGHI', 'BCUT2D_CHGLO', 'BCUT2D_LOGPHI', 'BCUT2D_LOGPLOW',
                      'BCUT2D_MRHI', 'BCUT2D_MRLOW'])
descriptors2 = descriptors2.dropna()

X = descriptors2.drop(['Id','SMILES','Expected'], axis=1)


y = descriptors2[['Expected']]

In [10]:
X['assay'] = X['assay'].astype("int64")

X_train, X_test, y_train, y_test = train_test_split(X[boruta_features2], y['Expected'].map({2:0, 1:1}), test_size=0.2, random_state=0, stratify=y['Expected'].map({2:0, 1:1}))

In [11]:
# descriptors2[['assay']] = descriptors2[['assay']].astype('int64')

# params = { 'max_depth': [6],
#            'n_estimators': [600],}

# xgbr = XGBClassifier(seed = 20)
# clf = GridSearchCV(estimator=xgbr, 
#                    param_grid=params,
#                    scoring='f1', 
#                    verbose=1,
#                    )

# clf.fit(descriptors2[boruta_features2], y['Expected'].map({2:0, 1:1}))

# print("Best parameters:", clf.best_params_)
# print("Lowest error: ", (-clf.best_score_)**(1/2.0))

In [33]:
descriptors2['assay'] = descriptors2['assay'].astype('int64') 
model = XGBClassifier(seed = 20, max_depth=10, n_estimators=700 )
model.fit(descriptors2[boruta_features2], y['Expected'].map({2:0, 1:1}))

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=10, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=700, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [34]:
preds = model.predict(X_test)
f1_score(preds, y_test)

0.9988058275614999

In [35]:
test_processed['assay'] = test_processed['assay'].astype('int64')

test_preds = model.predict(test_processed[boruta_features2])
test_preds

array([0, 0, 1, ..., 0, 0, 0])

In [36]:
np.unique(test_preds, return_counts=True)

(array([0, 1]), array([9674, 1320]))

In [37]:
res = pd.DataFrame({})
res['Id'] = test_processed['x']
res['Predicted'] = test_preds
res['Predicted'] = res['Predicted'].map({0:2, 1:1})
res

,Id,Predicted
0,CC1=CC(=C(C=C1)C(C)(C)C)O;1682,2
1,CC1=CC(=C(C=C1)C(C)(C)C)O;2451,2
2,CC1=CC(=C(C=C1)C(C)(C)C)O;2442,1
3,CC1=CC(=C(C=C1)C(C)(C)C)O;32,2
4,CC1=CC(=C(C=C1)C(C)(C)C)O;1382,2
...,...,...
10989,CC(=CC(=O)C)C;1856,2
10990,CCCCCCCCCC[N+](C)(C)CC1=CC=CC=C1.[Cl-];1848,1
10991,CC1=C(C(=O)N(N1C)C2=CC=CC=C2)N(C)CS(=O)(=O)[O-...,2
10992,COC1=CC=CC(=C1)C=O;2,2


In [38]:
# # res['Predicted2'].value_counts()
# res = res.drop(columns='Predicted')

In [39]:
from google.colab import files

res.to_csv('28-03-23-2.csv', encoding = 'utf-8-sig', index=False) 
files.download('28-03-23-2.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [210]:
res

,Id,Predicted2
0,CC1=CC(=C(C=C1)C(C)(C)C)O;1682,2
1,CC1=CC(=C(C=C1)C(C)(C)C)O;2451,2
2,CC1=CC(=C(C=C1)C(C)(C)C)O;2442,1
3,CC1=CC(=C(C=C1)C(C)(C)C)O;32,2
4,CC1=CC(=C(C=C1)C(C)(C)C)O;1382,2
...,...,...
10989,CC(=CC(=O)C)C;1856,2
10990,CCCCCCCCCC[N+](C)(C)CC1=CC=CC=C1.[Cl-];1848,1
10991,CC1=C(C(=O)N(N1C)C2=CC=CC=C2)N(C)CS(=O)(=O)[O-...,2
10992,COC1=CC=CC(=C1)C=O;2,2


In [40]:
morgan = df_processed[df_processed.columns[:260]]


In [50]:
morganB = morgan.drop(columns=['Id', 'Expected', 'SMILES'])
assays = descriptors.groupby('assay').count().sort_values(by='Id', ascending=False).head(10)['Id']

sample = morganB.loc[morganB['assay'].isin(assays.index.tolist())]
sample['Expected'] =  morgan['Expected']
sample

,assay,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,x248,x249,x250,x251,x252,x253,x254,x255,x256,Expected
17,1852,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
21,2,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
32,1857,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
51,2452,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
57,2451,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75372,1852,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
75373,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
75374,1852,1,0,0,0,0,1,0,0,1,...,0,0,0,0,1,0,0,0,0,1
75375,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2


167

In [52]:
model = XGBClassifier(max_depth=6, n_estimators=600, colsample_bytree=0.3)

boruta_features3 = []

# let's initialize Boruta
feat_selector = BorutaPy(
    verbose=2,
    estimator=model,
    n_estimators='auto',
    max_iter=10  # number of iterations to perform
)

# train Boruta
# N.B.: X and y must be numpy arrays
feat_selector.fit(np.array(sample.drop(columns='Expected')), np.array(sample['Expected'].map({2:0, 1:1})))

# print support and ranking for each feature
print("\n------Support and Ranking for each feature------")
for i in range(len(feat_selector.support_)):
    if feat_selector.support_[i]:
        boruta_features3.append(sample.columns[i])
        print("Passes the test: ", sample.columns[i],
              " - Ranking: ", feat_selector.ranking_[i])
    else:
        print("Doesn't pass the test: ",
              sample.columns[i], " - Ranking: ", feat_selector.ranking_[i])

Iteration: 	1 / 10
Confirmed: 	0
Tentative: 	257
Rejected: 	0
Iteration: 	2 / 10
Confirmed: 	0
Tentative: 	257
Rejected: 	0
Iteration: 	3 / 10
Confirmed: 	0
Tentative: 	257
Rejected: 	0
Iteration: 	4 / 10
Confirmed: 	0
Tentative: 	257
Rejected: 	0
Iteration: 	5 / 10
Confirmed: 	0
Tentative: 	257
Rejected: 	0
Iteration: 	6 / 10
Confirmed: 	0
Tentative: 	257
Rejected: 	0
Iteration: 	7 / 10
Confirmed: 	0
Tentative: 	257
Rejected: 	0
Iteration: 	8 / 10
Confirmed: 	0
Tentative: 	76
Rejected: 	181
Iteration: 	9 / 10
Confirmed: 	0
Tentative: 	76
Rejected: 	181


BorutaPy finished running.

Iteration: 	10 / 10
Confirmed: 	0
Tentative: 	10
Rejected: 	181

------Support and Ranking for each feature------
Doesn't pass the test:  assay  - Ranking:  2
Doesn't pass the test:  x1  - Ranking:  212
Doesn't pass the test:  x2  - Ranking:  246
Doesn't pass the test:  x3  - Ranking:  79
Doesn't pass the test:  x4  - Ranking:  31
Doesn't pass the test:  x5  - Ranking:  230
Doesn't pass the test:  x6  - Ran

In [61]:
fingerprints = []
for i, j in zip(sample.columns, feat_selector.ranking_):
  if feat_selector.ranking_[j] < 180:
    fingerprints.append(sample.columns[j])
    print(sample.columns[j], feat_selector.ranking_[j])

x212 36
x246 36
x79 52
x230 42
x73 33
x55 18
x29 112
x85 18
x150 43
x183 162
x82 2
x175 95
x47 121
x94 69
x186 49
x4 31
x241 73
x192 131
x108 168
x75 14
x224 76
x115 105
x33 66
x93 163
x183 162
x226 97
x211 63
x202 44
x30 108
x185 123
x160 14
x115 105
x18 2
x170 144
x138 44
x244 160
x202 44
x189 21
x25 81
x24 4
x188 173
x247 100
x208 96
x78 38
x165 6
x87 84
x193 18
x33 66
x235 158
x85 18
x127 83
x38 93
x155 10
x239 173
x107 140
x238 130
x18 2
x3 79
x51 61
x142 168
x178 71
x141 138
x163 67
x69 165
x201 112
x210 119
x91 178
x100 57
x57 138
x47 121
x189 21
x204 132
x166 171
x178 71
x140 80
x168 157
x63 24
x182 150
x235 158
x18 2
x105 166
x50 2
x88 51
x144 155
x195 41
x36 115
x117 2
x248 73
x83 107
x242 16
x54 115
x175 95
x186 49
x106 178
x240 12
x197 59
x193 18
x229 63
x80 155
x138 44
x168 157
x22 125
x155 10
x245 59
x216 102
x33 66
x223 163
x38 93
x43 103
x70 87
x7 55
x68 78
x119 88
x98 58
x124 117
x217 127
x209 173
x67 2
x115 105
x6 73
x171 56
x90 142
x157 136
x199 168
x144 155
x56 170


In [62]:
fingerprints

['x212',
 'x246',
 'x79',
 'x230',
 'x73',
 'x55',
 'x29',
 'x85',
 'x150',
 'x183',
 'x82',
 'x175',
 'x47',
 'x94',
 'x186',
 'x4',
 'x241',
 'x192',
 'x108',
 'x75',
 'x224',
 'x115',
 'x33',
 'x93',
 'x183',
 'x226',
 'x211',
 'x202',
 'x30',
 'x185',
 'x160',
 'x115',
 'x18',
 'x170',
 'x138',
 'x244',
 'x202',
 'x189',
 'x25',
 'x24',
 'x188',
 'x247',
 'x208',
 'x78',
 'x165',
 'x87',
 'x193',
 'x33',
 'x235',
 'x85',
 'x127',
 'x38',
 'x155',
 'x239',
 'x107',
 'x238',
 'x18',
 'x3',
 'x51',
 'x142',
 'x178',
 'x141',
 'x163',
 'x69',
 'x201',
 'x210',
 'x91',
 'x100',
 'x57',
 'x47',
 'x189',
 'x204',
 'x166',
 'x178',
 'x140',
 'x168',
 'x63',
 'x182',
 'x235',
 'x18',
 'x105',
 'x50',
 'x88',
 'x144',
 'x195',
 'x36',
 'x117',
 'x248',
 'x83',
 'x242',
 'x54',
 'x175',
 'x186',
 'x106',
 'x240',
 'x197',
 'x193',
 'x229',
 'x80',
 'x138',
 'x168',
 'x22',
 'x155',
 'x245',
 'x216',
 'x33',
 'x223',
 'x38',
 'x43',
 'x70',
 'x7',
 'x68',
 'x119',
 'x98',
 'x124',
 'x217',
 'x

In [41]:
var = VarianceThresh()
morganV, sel = var.fit_transform(morgan)
morganV

,Id,Expected,assay,SMILES,x2,x34,x39,x40,x50,x65,...,x137,x139,x148,x159,x168,x176,x184,x187,x215,x252
0,C1=CC(=CC=C1C(C2=CC=C(C=C2)O)C(Cl)(Cl)Cl)O;1644,2,1644,Oc1ccc(C(c2ccc(O)cc2)C(Cl)(Cl)Cl)cc1,1,1,0,1,0,0,...,0,0,1,0,0,1,0,0,1,0
1,C1=CC(=CC=C1C(C2=CC=C(C=C2)O)C(Cl)(Cl)Cl)O;1630,2,1630,Oc1ccc(C(c2ccc(O)cc2)C(Cl)(Cl)Cl)cc1,1,1,0,1,0,0,...,0,0,1,0,0,1,0,0,1,0
2,C1=CC(=CC=C1C(C2=CC=C(C=C2)O)C(Cl)(Cl)Cl)O;29,2,29,Oc1ccc(C(c2ccc(O)cc2)C(Cl)(Cl)Cl)cc1,1,1,0,1,0,0,...,0,0,1,0,0,1,0,0,1,0
3,C1=CC(=CC=C1C(C2=CC=C(C=C2)O)C(Cl)(Cl)Cl)O;1618,2,1618,Oc1ccc(C(c2ccc(O)cc2)C(Cl)(Cl)Cl)cc1,1,1,0,1,0,0,...,0,0,1,0,0,1,0,0,1,0
4,C1=CC(=CC=C1C(C2=CC=C(C=C2)O)C(Cl)(Cl)Cl)O;1638,2,1638,Oc1ccc(C(c2ccc(O)cc2)C(Cl)(Cl)Cl)cc1,1,1,0,1,0,0,...,0,0,1,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75372,C1=CC=C(C=C1)NC(=S)N;1852,1,1852,NC(=S)Nc1ccccc1,0,0,0,1,0,1,...,0,0,1,0,0,1,0,0,1,0
75373,CN1CN(CN(C1)C)C;2,2,2,CN1CN(C)CN(C)C1,0,1,0,0,0,0,...,0,0,0,1,1,0,0,1,0,0
75374,CCCCC1CCC(=O)O1;1852,1,1852,CCCCC1CCC(=O)O1,0,1,1,0,0,1,...,0,1,0,1,1,0,0,0,0,1
75375,CCOC(=O)CCC1=CC=CC=C1;2,2,2,CCOC(=O)CCc1ccccc1,0,1,1,1,0,1,...,0,1,1,0,0,1,1,1,1,0


In [42]:
morganV['assay'] = morganV['assay'].astype('int64')

model2 = XGBClassifier(seed = 20, max_depth=10, n_estimators=700)
model2.fit(morganV.drop(columns=['Id', 'SMILES', 'Expected']), morganV['Expected'].map({2:0, 1:1}))

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=10, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=700, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [ ]:
morgan['assay'] = morgan['assay'].astype('int64')
morganBoruta = morgan[fingerprints]

morganBoruta['assay'] = morgan['assay']

model3 = XGBClassifier(seed = 20, max_depth=10, n_estimators=700)
model3.fit(morgan.drop(columns=['Id', 'SMILES', 'Expected']), morganV['Expected'].map({2:0, 1:1}))

In [43]:
test_preds2 = model2.predict(test_processed[morganV.columns[2:]].drop(columns='SMILES'))
test_preds2

array([0, 0, 0, ..., 0, 0, 0])

In [44]:
np.unique(test_preds2, return_counts=True)

(array([0, 1]), array([9616, 1378]))

In [46]:
res2 = pd.DataFrame({})
res2['Id'] = test_processed['x']
res2['Predicted'] = test_preds2
res2['Predicted'] = res2['Predicted'].map({0:2, 1:1})
res2

,Id,Predicted
0,CC1=CC(=C(C=C1)C(C)(C)C)O;1682,2
1,CC1=CC(=C(C=C1)C(C)(C)C)O;2451,2
2,CC1=CC(=C(C=C1)C(C)(C)C)O;2442,2
3,CC1=CC(=C(C=C1)C(C)(C)C)O;32,2
4,CC1=CC(=C(C=C1)C(C)(C)C)O;1382,2
...,...,...
10989,CC(=CC(=O)C)C;1856,2
10990,CCCCCCCCCC[N+](C)(C)CC1=CC=CC=C1.[Cl-];1848,1
10991,CC1=C(C(=O)N(N1C)C2=CC=CC=C2)N(C)CS(=O)(=O)[O-...,2
10992,COC1=CC=CC(=C1)C=O;2,2


In [47]:
res2.Predicted.value_counts()

2    9616
1    1378
Name: Predicted, dtype: int64

In [48]:


from google.colab import files

res2.to_csv('28-03-23-m1.csv', encoding = 'utf-8-sig', index=False) 
files.download('28-03-23-m1.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [257]:
maccs = FingerprintGeneratorM()
df_maccs = maccs.fit_transform(df_processed[['Id', 'SMILES', 'assay', 'Expected']])
df_maccs

FingerprintGenerator done


,Id,SMILES,assay,Expected,x1,x2,x3,x4,x5,x6,...,x158,x159,x160,x161,x162,x163,x164,x165,x166,x167
0,C1=CC(=CC=C1C(C2=CC=C(C=C2)O)C(Cl)(Cl)Cl)O;1644,Oc1ccc(C(c2ccc(O)cc2)C(Cl)(Cl)Cl)cc1,1644,2,0,0,0,0,0,0,...,1,0,1,0,0,1,1,1,1,0
1,C1=CC(=CC=C1C(C2=CC=C(C=C2)O)C(Cl)(Cl)Cl)O;1630,Oc1ccc(C(c2ccc(O)cc2)C(Cl)(Cl)Cl)cc1,1630,2,0,0,0,0,0,0,...,1,0,1,0,0,1,1,1,1,0
2,C1=CC(=CC=C1C(C2=CC=C(C=C2)O)C(Cl)(Cl)Cl)O;29,Oc1ccc(C(c2ccc(O)cc2)C(Cl)(Cl)Cl)cc1,29,2,0,0,0,0,0,0,...,1,0,1,0,0,1,1,1,1,0
3,C1=CC(=CC=C1C(C2=CC=C(C=C2)O)C(Cl)(Cl)Cl)O;1618,Oc1ccc(C(c2ccc(O)cc2)C(Cl)(Cl)Cl)cc1,1618,2,0,0,0,0,0,0,...,1,0,1,0,0,1,1,1,1,0
4,C1=CC(=CC=C1C(C2=CC=C(C=C2)O)C(Cl)(Cl)Cl)O;1638,Oc1ccc(C(c2ccc(O)cc2)C(Cl)(Cl)Cl)cc1,1638,2,0,0,0,0,0,0,...,1,0,1,0,0,1,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75372,C1=CC=C(C=C1)NC(=S)N;1852,NC(=S)Nc1ccccc1,1852,1,0,0,0,0,0,0,...,0,1,0,0,1,1,1,0,1,0
75373,CN1CN(CN(C1)C)C;2,CN1CN(C)CN(C)C1,2,2,0,0,0,0,0,0,...,0,1,0,1,1,0,1,0,1,0
75374,CCCCC1CCC(=O)O1;1852,CCCCC1CCC(=O)O1,1852,1,0,0,0,0,0,0,...,1,0,1,1,0,0,0,1,1,0
75375,CCOC(=O)CCC1=CC=CC=C1;2,CCOC(=O)CCc1ccccc1,2,2,0,0,0,0,0,0,...,1,0,1,1,0,1,1,1,1,0


In [ ]:
df_maccs = df_maccs.drop(columns=['Id', 'Expected', 'SMILES'])
assays = descriptors.groupby('assay').count().sort_values(by='Id', ascending=False).head(10)['Id']

sample = df_maccs.loc[df_maccs['assay'].isin(assays.index.tolist())]
sample['Expected'] =  morgan['Expected']

model = XGBClassifier(max_depth=6, n_estimators=600, colsample_bytree=0.3)

boruta_features4 = []

# let's initialize Boruta
feat_selector = BorutaPy(
    verbose=2,
    estimator=model,
    n_estimators='auto',
    max_iter=10  # number of iterations to perform
)

# train Boruta
# N.B.: X and y must be numpy arrays
feat_selector.fit(np.array(sample.drop(columns='Expected')), np.array(sample['Expected'].map({2:0, 1:1})))

# print support and ranking for each feature
print("\n------Support and Ranking for each feature------")
for i in range(len(feat_selector.support_)):
    if feat_selector.support_[i]:
        boruta_features4.append(sample.columns[i])
        print("Passes the test: ", sample.columns[i],
              " - Ranking: ", feat_selector.ranking_[i])
    else:
        print("Doesn't pass the test: ",
              sample.columns[i], " - Ranking: ", feat_selector.ranking_[i])

In [259]:
variance = VarianceThresh()
df_maccsV, s = variance.fit_transform(df_maccs)
df_maccsV

,Id,Expected,assay,SMILES,x66,x67,x75,x78,x81,x82,...,x156,x157,x158,x159,x160,x161,x162,x163,x164,x166
0,C1=CC(=CC=C1C(C2=CC=C(C=C2)O)C(Cl)(Cl)Cl)O;1644,2,1644,Oc1ccc(C(c2ccc(O)cc2)C(Cl)(Cl)Cl)cc1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,1,1,1
1,C1=CC(=CC=C1C(C2=CC=C(C=C2)O)C(Cl)(Cl)Cl)O;1630,2,1630,Oc1ccc(C(c2ccc(O)cc2)C(Cl)(Cl)Cl)cc1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,1,1,1
2,C1=CC(=CC=C1C(C2=CC=C(C=C2)O)C(Cl)(Cl)Cl)O;29,2,29,Oc1ccc(C(c2ccc(O)cc2)C(Cl)(Cl)Cl)cc1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,1,1,1
3,C1=CC(=CC=C1C(C2=CC=C(C=C2)O)C(Cl)(Cl)Cl)O;1618,2,1618,Oc1ccc(C(c2ccc(O)cc2)C(Cl)(Cl)Cl)cc1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,1,1,1
4,C1=CC(=CC=C1C(C2=CC=C(C=C2)O)C(Cl)(Cl)Cl)O;1638,2,1638,Oc1ccc(C(c2ccc(O)cc2)C(Cl)(Cl)Cl)cc1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75372,C1=CC=C(C=C1)NC(=S)N;1852,1,1852,NC(=S)Nc1ccccc1,0,0,0,1,0,1,...,0,1,0,1,0,0,1,1,1,1
75373,CN1CN(CN(C1)C)C;2,2,2,CN1CN(C)CN(C)C1,0,0,0,1,1,0,...,0,0,0,1,0,1,1,0,1,1
75374,CCCCC1CCC(=O)O1;1852,1,1852,CCCCC1CCC(=O)O1,0,0,0,0,0,0,...,1,0,1,0,1,1,0,0,0,1
75375,CCOC(=O)CCC1=CC=CC=C1;2,2,2,CCOC(=O)CCc1ccccc1,0,0,0,0,0,0,...,1,0,1,0,1,1,0,1,1,1


In [260]:
df_maccsV['assay'] = df_maccsV['assay'].astype('int64')

model3 = XGBClassifier(seed = 20, max_depth=6, n_estimators=600)
model3.fit(df_maccsV.drop(columns=['Id', 'SMILES', 'Expected']), df_maccsV['Expected'].map({2:0, 1:1}))

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=600, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [266]:
test_preds3 = model3.predict(test_processed[df_maccsV.columns[2:]].drop(columns='SMILES'))
np.unique(test_preds3, return_counts=True)

(array([0, 1]), array([10304,   690]))

In [262]:
res3 = pd.DataFrame({})
res3['Id'] = test_processed['x']
res3['Predicted'] = test_preds3
res3['Predicted2'] = res3['Predicted'].map({0:2, 1:1})
res3

,Id,Predicted,Predicted2
0,CC1=CC(=C(C=C1)C(C)(C)C)O;1682,0,2
1,CC1=CC(=C(C=C1)C(C)(C)C)O;2451,0,2
2,CC1=CC(=C(C=C1)C(C)(C)C)O;2442,0,2
3,CC1=CC(=C(C=C1)C(C)(C)C)O;32,0,2
4,CC1=CC(=C(C=C1)C(C)(C)C)O;1382,0,2
...,...,...,...
10989,CC(=CC(=O)C)C;1856,0,2
10990,CCCCCCCCCC[N+](C)(C)CC1=CC=CC=C1.[Cl-];1848,0,2
10991,CC1=C(C(=O)N(N1C)C2=CC=CC=C2)N(C)CS(=O)(=O)[O-...,0,2
10992,COC1=CC=CC(=C1)C=O;2,0,2


In [264]:
res3 = res3.drop(columns='Predicted')

from google.colab import files

res3.to_csv('27-03-23-m2.csv', encoding = 'utf-8-sig', index=False) 
files.download('27-03-23-m2.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Boruta

In [ ]:
model = XGBClassifier(max_depth=6, learning_rate=0.01, n_estimators=600, colsample_bytree=0.3)

boruta_features = []

# let's initialize Boruta
feat_selector = BorutaPy(
    verbose=2,
    estimator=model,
    n_estimators='auto',
    max_iter=10  # number of iterations to perform
)

# train Boruta
# N.B.: X and y must be numpy arrays
feat_selector.fit(np.array(X_train), np.array(y_train))

# print support and ranking for each feature
print("\n------Support and Ranking for each feature------")
for i in range(len(feat_selector.support_)):
    if feat_selector.support_[i]:
        boruta_features.append(X_train.columns[i])
        print("Passes the test: ", X_train.columns[i],
              " - Ranking: ", feat_selector.ranking_[i])
    else:
        print("Doesn't pass the test: ",
              X_train.columns[i], " - Ranking: ", feat_selector.ranking_[i])

Iteration: 	1 / 10
Confirmed: 	0
Tentative: 	201
Rejected: 	0
Iteration: 	2 / 10
Confirmed: 	0
Tentative: 	201
Rejected: 	0
Iteration: 	3 / 10
Confirmed: 	0
Tentative: 	201
Rejected: 	0
Iteration: 	4 / 10
Confirmed: 	0
Tentative: 	201
Rejected: 	0
Iteration: 	5 / 10
Confirmed: 	0
Tentative: 	201
Rejected: 	0
Iteration: 	6 / 10
Confirmed: 	0
Tentative: 	201
Rejected: 	0
Iteration: 	7 / 10
Confirmed: 	0
Tentative: 	201
Rejected: 	0
Iteration: 	8 / 10
Confirmed: 	35
Tentative: 	133
Rejected: 	33
Iteration: 	9 / 10
Confirmed: 	35
Tentative: 	133
Rejected: 	33


BorutaPy finished running.

Iteration: 	10 / 10
Confirmed: 	35
Tentative: 	117
Rejected: 	33

------Support and Ranking for each feature------
Passes the test:  assayE  - Ranking:  1
Doesn't pass the test:  MaxEStateIndex  - Ranking:  2
Doesn't pass the test:  MinEStateIndex  - Ranking:  2
Doesn't pass the test:  MaxAbsEStateIndex  - Ranking:  2
Doesn't pass the test:  MinAbsEStateIndex  - Ranking:  2
Doesn't pass the test:  qed  - 

In [ ]:
X_train[boruta_features]

,assayE,HeavyAtomMolWt,MaxPartialCharge,MinAbsPartialCharge,BertzCT,Chi1,Chi1n,Chi2v,Chi3n,Chi3v,...,fr_C_O,fr_C_O_noCOO,fr_amide,fr_benzene,fr_phenol,fr_phenol_noOrthoHbond,fr_sulfonamd,fr_thiazole,fr_thiophene,fr_urea
36273,129,-1.147456,-0.473092,-1.285070,-1.302433,-1.596246,-1.641372,-1.045153,-1.401506,-1.492063,...,-0.802744,-0.729312,-0.490895,-0.979226,-0.187318,-0.186438,-0.214016,-0.096307,-0.084599,-0.235384
38323,41,0.187954,0.305449,0.829750,-0.111443,0.059418,0.185731,1.053573,-0.374865,0.423096,...,-0.802744,-0.729312,-0.490895,-0.979226,-0.187318,-0.186438,-0.214016,-0.096307,-0.084599,-0.235384
35554,146,0.984623,0.162577,0.564432,0.832542,1.123464,1.169582,0.710780,0.988228,0.693769,...,2.437535,2.706231,2.189204,0.131763,-0.187318,-0.186438,-0.214016,-0.096307,-0.084599,-0.235384
67207,13,-0.401553,2.713573,1.963518,-0.149954,-0.731718,-0.852962,-0.471086,-0.635846,-0.285705,...,0.277349,0.415869,0.849154,0.131763,-0.187318,-0.186438,4.345557,-0.096307,-0.084599,-0.235384
62773,126,0.226130,0.144513,0.530886,0.854176,0.350209,0.221514,0.590345,0.297447,0.441847,...,0.277349,0.415869,-0.490895,1.242753,-0.187318,-0.186438,-0.214016,-0.096307,-0.084599,-0.235384
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24503,75,0.226349,-0.639703,-0.925428,0.461766,0.374946,0.281338,1.416662,0.470185,1.849797,...,-0.802744,-0.729312,-0.490895,1.242753,-0.187318,-0.186438,-0.214016,-0.096307,-0.084599,-0.235384
20296,139,-0.262019,-0.311328,-0.315624,0.056727,-0.173988,-0.218765,-0.422359,-0.205442,-0.354974,...,-0.802744,-0.729312,-0.490895,0.131763,-0.187318,-0.186438,-0.214016,-0.096307,-0.084599,-0.235384
32329,21,0.165373,0.143869,0.529691,0.345756,0.518303,0.517287,-0.083548,0.184306,-0.122634,...,1.357442,1.561050,-0.490895,1.242753,-0.187318,-0.186438,-0.214016,-0.096307,-0.084599,-0.235384
42276,78,-0.922780,-0.710349,-1.056620,-0.599541,-0.776580,-0.550374,-0.360377,-0.462075,-0.680816,...,-0.802744,-0.729312,-0.490895,0.131763,1.125137,1.132688,-0.214016,-0.096307,-0.084599,-0.235384


## VIF

In [ ]:
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif_info = pd.DataFrame()
vif_info['VIF'] = [variance_inflation_factor(X_train[boruta_features].values, i) for i in range(X_train[boruta_features].shape[1])]
vif_info['Column'] = X_train[boruta_features].columns
vif_info.sort_values('VIF', ascending=False)

,VIF,Column
1,422359.203113,MolWt
3,316424.304741,ExactMolWt
2,98764.315258,HeavyAtomMolWt
31,14179.530156,HeavyAtomCount
4,4284.055024,NumValenceElectrons
8,2694.242248,Chi0n
10,2465.774451,Chi1
9,1984.227269,Chi0v
20,1507.509826,LabuteASA
11,1227.723459,Chi1n


In [ ]:
X = X_train[boruta_features].copy()
# X['HeavyAtomAvgWt'] = X['HeavyAtomMolWt'] / X['HeavyAtomCount']
# X['MolWt_HeavyAtomMolWt_diff'] = X['MolWt'] - X['HeavyAtomMolWt']

X = X.drop(['ExactMolWt', 'Chi1', 'Chi4n', 'Chi2v', 'HeavyAtomCount', 'HeavyAtomMolWt', 'Chi0v', 'Chi1n', 
            'fr_phenol_noOrthoHbond', 'Chi2n', 'Chi0', 'LabuteASA', 'MolMR', 'Chi1v', 'Chi4v', 'SlogP_VSA6', 'Chi3n'], axis=1)


vif_info = pd.DataFrame()
vif_info['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif_info['Column'] = X.columns
vif_info.sort_values('VIF', ascending=False)

final_cols = X.columns

In [ ]:
vif_info['VIF'].unique()[1]

24.730111166707044

In [ ]:
vif_info.sort_values('VIF', ascending=False)

,VIF,Column
2,157.205422,NumValenceElectrons
5,130.365159,Chi0n
1,24.730111,MolWt
4,16.255149,BertzCT
18,11.906435,fr_benzene
16,10.246243,VSA_EState6
10,9.412649,SMR_VSA7
6,4.223124,Chi3v
8,3.942029,PEOE_VSA6
9,3.872187,PEOE_VSA7


In [ ]:
final_cols.size

20

# Model Training

In [ ]:
model = RandomForestClassifier(n_estimators=400, max_depth=5, random_state=42)


In [ ]:
from sklearn.model_selection import RandomizedSearchCV

random_grid = {'bootstrap': [True, False],
 'max_depth': [3, 4, 5, 10, 20, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [150]}

In [ ]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune

rf = RandomForestClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores

rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 30, cv = 3, verbose=2, random_state=42, n_jobs = -1, scoring='f1')

# Fit the random search model
rf_random.fit(X_train[final_cols], y_train)

Fitting 3 folds for each of 30 candidates, totalling 90 fits


RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(), n_iter=30,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [3, 4, 5, 10, 20, None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [150]},
                   random_state=42, scoring='f1', verbose=2)

In [ ]:
base_model = RandomForestClassifier(n_estimators = 100, random_state = 42)
base_model.fit(X_train[final_cols], y_train)



array([1, 2, 2, ..., 2, 2, 2])

In [ ]:
preds = base_model.predict(X_test[final_cols])
print(f"acc: {base_model.score(X_test[final_cols], y_test)}  F1: {f1_score(preds, y_test.values)}")

acc: 0.8475467289719626  F1: 0.5263157894736843


In [ ]:
best_random = RandomForestClassifier(bootstrap=False, max_features='auto', min_samples_leaf=2,
                       n_estimators=500)

best_random.fit(X_train[final_cols], y_train)

RandomForestClassifier(bootstrap=False, max_features='auto', min_samples_leaf=2,
                       n_estimators=500)

In [ ]:
preds2 = best_random.predict(X_test[final_cols])
print(f"acc: {best_random.score(X_test[final_cols], y_test)}  F1: {f1_score(preds2, y_test.values)}")

acc: 0.8539719626168224  F1: 0.5410036719706243


In [ ]:
rf_random.best_estimator_

RandomForestClassifier(bootstrap=False, max_features='auto', min_samples_leaf=2,
                       n_estimators=150)

In [ ]:
grid = {'solver': ['newton-cg', 'lbfgs', 'liblinear'],
 'penalty': ['l2'],
 'C': [100, 10, 1.0, 0.1, 0.01]}

In [ ]:
xgb = XGBClassifier(learning_rate=0.02, n_estimators=600, objective='binary:logistic',
                    silent=True, nthread=1)

params = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5]
        }


In [ ]:
folds = 3
param_comb = 5

y2 = y_train['Expected'].map({2:0, 1:1})


random_search = RandomizedSearchCV(xgb, param_distributions=params, n_iter=30, scoring='f1', n_jobs=-1, cv=3, verbose=3, random_state=42)

# Here we go
random_search.fit(X_train[final_cols], y2)


Fitting 3 folds for each of 30 candidates, totalling 90 fits
[05:24:26] WARNING: ../src/learner.cc:767: 
Parameters: { "silent" } are not used.



RandomizedSearchCV(cv=3,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric=None, feature_types=None,
                                           gamma=None, gpu_id=None,
                                           grow_policy=None,
                                           importance_type=None,
                                           interaction_constraints=None,
                                           learning_rate...
                                           min_child_weight=None, missing=nan,
                                           monotone_constraints=None,
                                           n_estimators=600, n_jobs=None,
                                           nthread=1, num_parallel_tree=None,
                                           predictor=None, ...),
                   n_iter=30, n_jobs=-1,
                   param_distributions={'colsample_bytree': [0.6, 0.8, 1.0],
                                        'gamma': [0.5, 1, 1.5, 2, 5],
                                        'max_depth': [3, 4, 5],
                                        'min_child_weight': [1, 5, 10],
                                        'subsample': [0.6, 0.8, 1.0]},
                   random_state=42, scoring='f1', verbose=3)

In [ ]:
print('\n All results:')
print(random_search.cv_results_)
print('\n Best estimator:')
print(random_search.best_estimator_)
print('\n Best normalized gini score for %d-fold search with %d parameter combinations:' % (folds, param_comb))
print(random_search.best_score_ * 2 - 1)
print('\n Best hyperparameters:')
print(random_search.best_params_)
results = pd.DataFrame(random_search.cv_results_)


 All results:
{'mean_fit_time': array([10.16115546,  8.77746518, 10.67506949,  6.90329885,  8.98260784,
       10.94896873,  8.70946479,  9.97864501, 13.7814045 , 14.22631407,
       13.5622824 , 11.3408165 , 13.92403873, 13.85271637, 14.58024788,
       11.67534892, 10.49363724, 11.09710741,  9.5321242 ,  8.78274425,
        7.22162461, 17.00099389, 11.91968862, 11.21867164, 16.96571819,
        9.91310175,  7.36362449, 14.41711275, 11.71606239,  7.72759302]), 'std_fit_time': array([1.49837319, 0.73191483, 0.37804566, 0.70478389, 0.46619737,
       0.39089363, 0.57849137, 0.80692173, 0.03585325, 0.06963627,
       0.04899856, 0.04029885, 0.080637  , 0.15795491, 0.1323492 ,
       0.0498088 , 0.73716468, 0.28908161, 0.88567855, 0.87123845,
       0.16671459, 0.49070506, 0.09154688, 0.68961674, 0.6754089 ,
       0.82486739, 0.83118042, 0.08349249, 0.43596048, 0.83123931]), 'mean_score_time': array([0.06835143, 0.04691664, 0.06232667, 0.05607979, 0.06276615,
       0.10885557, 0.068347

In [ ]:
xgbest = XGBClassifier(learning_rate=0.02, n_estimators=600, objective='binary:logistic',
                    silent=True, nthread=1, subsample=1.0, min_child_weight=5, max_depth=5, gamma=1, colsample_bytree=1.0)

xgbest.fit(X_train[final_cols], y2)

[05:33:44] WARNING: ../src/learner.cc:767: 
Parameters: { "silent" } are not used.



XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=1.0, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=1, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.02, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=5, missing=nan, monotone_constraints=None,
              n_estimators=600, n_jobs=None, nthread=1, num_parallel_tree=None,
              predictor=None, ...)

In [ ]:
predsxg = xgbest.predict(X_test[final_cols])

print(xgbest.score(X_test[final_cols], y_test['Expected'].map({2:0, 1:1})))

0.8603971962616822


In [ ]:
print(f1_score(predsxg, y_test['Expected'].map({2:0, 1:1})))

0.5447619047619048


Test Data

In [ ]:
X = descriptors_dropna.drop(['Id','SMILES','Expected'], axis=1)
y = descriptors_dropna[['Expected']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0, stratify=y['Expected'])

#Encoding
encoder = LabelEncoder()
X_train['assayE'] = encoder.fit_transform(X_train[['assay']])
X_test['assayE'] = encoder.transform(X_test[['assay']])

# Scaling
scaler = StandardScaler()
X_train_scaled = X_train.drop(columns=['assay', 'assayE'])
X_test_scaled = X_test.drop(columns=['assay', 'assayE'])

# X_train_scaled = X_train.copy()
# X_test_scaled = X_test.copy()

X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train_scaled), columns=X_train_scaled.columns)
X_test_scaled = pd.DataFrame(scaler.transform(X_test_scaled), columns=X_test_scaled.columns)


X_train = pd.concat([X_train[['assayE']].reset_index(drop=True), X_train_scaled], axis=1)
X_test = pd.concat([X_test[['assayE']].reset_index(drop=True), X_test_scaled], axis=1)

# X_train = X_train_scaled
# X_test = X_test_scaled
X_train

,assayE,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,142,-0.394303,0.787853,-0.394303,-0.077125,-0.314256,-1.177470,-1.176786,-1.176099,-1.068749,...,-0.231036,-0.215466,-0.09226,-0.100057,-0.038723,-0.096918,-0.054076,-0.087291,-0.279316,-0.23324
1,99,-1.715306,0.385754,-1.715306,0.318235,-0.238270,-1.012543,-1.049978,-1.009479,-0.832214,...,-0.231036,-0.215466,-0.09226,-0.100057,-0.038723,-0.096918,-0.054076,-0.087291,-0.279316,-0.23324
2,89,0.667196,0.597094,0.667196,-0.517074,0.982431,-0.017466,-0.062607,-0.018697,0.074504,...,-0.231036,-0.215466,-0.09226,-0.100057,-0.038723,-0.096918,-0.054076,-0.087291,-0.279316,-0.23324
3,21,0.021885,0.368194,0.021885,-0.495348,-0.864631,-0.852548,-0.829370,-0.850820,-0.753369,...,-0.231036,-0.215466,-0.09226,-0.100057,-0.038723,-0.096918,-0.054076,-0.087291,-0.279316,-0.23324
4,126,1.288825,-0.119480,1.288825,-0.558434,-2.897808,6.276056,6.169027,6.281638,6.421531,...,-0.231036,-0.215466,-0.09226,-0.100057,-0.038723,-0.096918,-0.054076,-0.087291,-0.279316,-0.23324
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52758,116,1.220464,-1.794416,1.220464,-0.459721,-0.581134,2.176529,2.094864,2.180536,2.518701,...,-0.231036,-0.215466,-0.09226,-0.100057,-0.038723,-0.096918,-0.054076,-0.087291,-0.279316,-0.23324
52759,51,1.137049,-2.085181,1.137049,-0.515460,-0.140013,0.795484,0.825641,0.798089,0.823532,...,3.609584,-0.215466,-0.09226,-0.100057,-0.038723,-0.096918,-0.054076,-0.087291,-0.279316,-0.23324
52760,140,0.103058,-0.902960,0.103058,-0.538617,-0.715350,0.036446,0.054671,0.038420,-0.043763,...,-0.231036,-0.215466,-0.09226,-0.100057,-0.038723,-0.096918,-0.054076,-0.087291,-0.279316,-0.23324
52761,13,0.057555,-0.004893,0.057555,-0.400462,0.668673,-0.680336,-0.611208,-0.685231,-0.871636,...,-0.231036,-0.215466,-0.09226,-0.100057,-0.038723,-0.096918,-0.054076,-0.087291,-0.279316,-0.23324


In [ ]:
final_cols2 = ['assay', 'MolWt', 'NumValenceElectrons', 'BalabanJ', 'BertzCT',
       'Chi0n', 'Chi3v', 'Kappa1', 'PEOE_VSA6', 'PEOE_VSA7', 'SMR_VSA7',
       'EState_VSA8', 'VSA_EState10', 'VSA_EState2', 'VSA_EState4',
       'VSA_EState5', 'VSA_EState6', 'MolLogP', 'fr_benzene', 'fr_phenol']

In [ ]:
y2 = y_train['Expected'].map({2:0, 1:1})

xgbest = XGBClassifier(learning_rate=0.02, n_estimators=600, objective='binary:logistic',
                    silent=True, nthread=1, subsample=1.0, min_child_weight=5, max_depth=5, gamma=1, colsample_bytree=1.0, random_state=42)

xgbest.fit(X_train[final_cols], y2)

[06:06:37] WARNING: ../src/learner.cc:767: 
Parameters: { "silent" } are not used.



XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=1.0, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=1, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.02, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=5, missing=nan, monotone_constraints=None,
              n_estimators=600, n_jobs=None, nthread=1, num_parallel_tree=None,
              predictor=None, ...)

In [ ]:
predsxg = xgbest.predict(X_test[final_cols])

print(xgbest.score(X_test[final_cols], y_test['Expected'].map({2:0, 1:1})))

0.8830812770849916


In [ ]:
print(f1_score(predsxg, y_test['Expected'].map({2:0, 1:1})))

0.3873957367933272


In [ ]:
best_random = RandomForestClassifier(bootstrap=False, max_features='auto', min_samples_leaf=2,
                       n_estimators=500, random_state=42)

best_random.fit(X_train[final_cols], y_train)

RandomForestClassifier(bootstrap=False, max_features='auto', min_samples_leaf=2,
                       n_estimators=500, random_state=42)

In [ ]:
preds2 = best_random.predict(X_test[final_cols])
print(f"acc: {best_random.score(X_test[final_cols], y_test)}  F1: {f1_score(preds2, y_test.values)}")

acc: 0.8565048200229946  F1: 0.41710077240883786


In [ ]:
log = LogisticRegression(random_state=1, class_weight={2:1,1:5})

log.fit(X_train[final_cols], y_train )

LogisticRegression(class_weight={1: 5, 2: 1}, random_state=1)

In [ ]:
preds3 = log.predict(X_test[final_cols])
print(f"acc: {log.score(X_test[final_cols], y_test)}  F1: {f1_score(preds3, y_test.values)}")

acc: 0.7042097815512515  F1: 0.28908491869486663


In [ ]:
X = descriptors_dropna.drop(['Id','SMILES','Expected'], axis=1)
y = descriptors_dropna[['Expected']]

#Encoding
encoder = LabelEncoder()
X['assayE'] = encoder.fit_transform(X[['assay']])

# Scaling
scaler = StandardScaler()
X_scaled = X.drop(columns=['assay', 'assayE'])


X_scaled = pd.DataFrame(scaler.fit_transform(X_scaled), columns=X_scaled.columns)

X_scaled = pd.concat([X[['assayE']].reset_index(drop=True), X_scaled], axis=1)


X_scaled

,assayE,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,62,-0.293881,-0.195150,-0.293881,-0.276932,1.216602,0.224803,0.274387,0.217035,-0.044591,...,-0.230411,-0.213255,-0.092446,-0.101257,-0.037703,-0.095977,-0.05484,-0.084309,-0.279355,-0.234539
1,48,-0.293881,-0.195150,-0.293881,-0.276932,1.216602,0.224803,0.274387,0.217035,-0.044591,...,-0.230411,-0.213255,-0.092446,-0.101257,-0.037703,-0.095977,-0.05484,-0.084309,-0.279355,-0.234539
2,131,-0.293881,-0.195150,-0.293881,-0.276932,1.216602,0.224803,0.274387,0.217035,-0.044591,...,-0.230411,-0.213255,-0.092446,-0.101257,-0.037703,-0.095977,-0.05484,-0.084309,-0.279355,-0.234539
3,36,-0.293881,-0.195150,-0.293881,-0.276932,1.216602,0.224803,0.274387,0.217035,-0.044591,...,-0.230411,-0.213255,-0.092446,-0.101257,-0.037703,-0.095977,-0.05484,-0.084309,-0.279355,-0.234539
4,56,-0.293881,-0.195150,-0.293881,-0.276932,1.216602,0.224803,0.274387,0.217035,-0.044591,...,-0.230411,-0.213255,-0.092446,-0.101257,-0.037703,-0.095977,-0.05484,-0.084309,-0.279355,-0.234539
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75372,93,-1.657276,0.809560,-1.657276,-0.004438,0.146656,-0.972162,-0.954179,-0.971152,-0.992583,...,-0.230411,-0.213255,-0.092446,-0.101257,-0.037703,-0.095977,-0.05484,-0.084309,-0.279355,-0.234539
75373,99,-2.649682,1.243133,-2.649682,1.461180,-0.679339,-1.138740,-1.181734,-1.137225,-0.953084,...,-0.230411,-0.213255,-0.092446,-0.101257,-0.037703,-0.095977,-0.05484,-0.084309,-0.279355,-0.234539
75374,93,0.134150,0.639323,0.134150,-0.532166,-0.037052,-1.044714,-1.075800,-1.043203,-0.874084,...,-0.230411,-0.213255,-0.092446,-0.101257,-0.037703,-0.095977,-0.05484,-0.084309,-0.279355,-0.234539
75375,99,0.268016,0.581028,0.268016,-0.335107,0.492404,-0.783914,-0.803129,-0.782289,-0.637086,...,-0.230411,-0.213255,-0.092446,-0.101257,-0.037703,-0.095977,-0.05484,-0.084309,-0.279355,-0.234539


In [ ]:
best_random = RandomForestClassifier(bootstrap=False, max_features='auto', min_samples_leaf=2,
                       n_estimators=500, random_state=42)

best_random.fit(X_scaled[final_cols], y)

RandomForestClassifier(bootstrap=False, max_features='auto', min_samples_leaf=2,
                       n_estimators=500, random_state=42)

In [ ]:
test_processed2 = pd.concat([test_processed[['x', 'assay']], test_processed[sample.columns[4:]]], axis=1)
test_processed2

,x,assay,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,CC1=CC(=C(C=C1)C(C)(C)C)O;1682,1682,9.626968,0.025579,9.626968,0.025579,0.624614,164.248,148.120,164.120115,...,0,0,0,0,0,0,0,0,0,0
1,CC1=CC(=C(C=C1)C(C)(C)C)O;2451,2451,9.626968,0.025579,9.626968,0.025579,0.624614,164.248,148.120,164.120115,...,0,0,0,0,0,0,0,0,0,0
2,CC1=CC(=C(C=C1)C(C)(C)C)O;2442,2442,9.626968,0.025579,9.626968,0.025579,0.624614,164.248,148.120,164.120115,...,0,0,0,0,0,0,0,0,0,0
3,CC1=CC(=C(C=C1)C(C)(C)C)O;32,32,9.626968,0.025579,9.626968,0.025579,0.624614,164.248,148.120,164.120115,...,0,0,0,0,0,0,0,0,0,0
4,CC1=CC(=C(C=C1)C(C)(C)C)O;1382,1382,9.626968,0.025579,9.626968,0.025579,0.624614,164.248,148.120,164.120115,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10989,CC(=CC(=O)C)C;1856,1856,10.177778,0.125000,10.177778,0.125000,0.454518,98.145,88.065,98.073165,...,0,0,0,0,0,0,0,0,0,0
10990,CCCCCCCCCC[N+](C)(C)CC1=CC=CC=C1.[Cl-];1848,1848,2.355405,0.000000,2.355405,0.000000,0.434780,311.941,277.669,311.237978,...,0,0,0,0,0,0,0,0,6,0
10991,CC1=C(C(=O)N(N1C)C2=CC=CC=C2)N(C)CS(=O)(=O)[O-...,1848,12.541948,-4.454731,12.541948,0.000000,0.420180,351.360,333.216,351.086486,...,0,0,0,0,0,0,0,0,0,0
10992,COC1=CC=CC(=C1)C=O;2,2,10.219078,0.638287,10.219078,0.638287,0.575816,136.150,128.086,136.052429,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
#Encoding
test_processed2['assayE'] = encoder.fit_transform(test_processed2[['assay']])


# Scaling
test_processed_scaled = scaler.transform(test_processed2.drop(columns=['assay', 'assayE', 'x']))
test_processed_scaled = pd.DataFrame(scaler.transform(test_processed_scaled), columns=test_processed2.drop(columns=['assay', 'assayE', 'x']).columns)


test_processed3 = pd.concat([test_processed2[['assayE']].reset_index(drop=True), test_processed_scaled], axis=1)


test_processed3

,assayE,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,79,-3.477170,1.008731,-3.477170,-1.499300,-1.455422,-2.080320,-2.052052,-2.079490,-2.033718,...,-1.114959,-1.188775,-1.001034,-1.11151,-1.039125,-1.105188,-0.665868,-1.091417,-0.423284,-1.208551
1,124,-3.477170,1.008731,-3.477170,-1.499300,-1.455422,-2.080320,-2.052052,-2.079490,-2.033718,...,-1.114959,-1.188775,-1.001034,-1.11151,-1.039125,-1.105188,-0.665868,-1.091417,-0.423284,-1.208551
2,117,-3.477170,1.008731,-3.477170,-1.499300,-1.455422,-2.080320,-2.052052,-2.079490,-2.033718,...,-1.114959,-1.188775,-1.001034,-1.11151,-1.039125,-1.105188,-0.665868,-1.091417,-0.423284,-1.208551
3,135,-3.477170,1.008731,-3.477170,-1.499300,-1.455422,-2.080320,-2.052052,-2.079490,-2.033718,...,-1.114959,-1.188775,-1.001034,-1.11151,-1.039125,-1.105188,-0.665868,-1.091417,-0.423284,-1.208551
4,14,-3.477170,1.008731,-3.477170,-1.499300,-1.455422,-2.080320,-2.052052,-2.079490,-2.033718,...,-1.114959,-1.188775,-1.001034,-1.11151,-1.039125,-1.105188,-0.665868,-1.091417,-0.423284,-1.208551
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10989,95,-3.415403,1.038660,-3.415403,-1.157310,-6.402587,-2.083783,-2.055491,-2.082959,-2.043860,...,-1.114959,-1.188775,-1.001034,-1.11151,-1.039125,-1.105188,-0.665868,-1.091417,-0.423284,-1.208551
10990,91,-4.292593,1.001031,-4.292593,-1.587286,-6.976640,-2.072583,-2.044634,-2.071762,-2.011875,...,-1.114959,-1.188775,-1.001034,-1.11151,-1.039125,-1.105188,-0.665868,-1.091417,1.169412,-1.208551
10991,91,-3.150288,-0.339990,-3.150288,-1.587286,-7.401282,-2.070518,-2.041453,-2.069669,-2.011875,...,-1.114959,-1.188775,-1.001034,-1.11151,-1.039125,-1.105188,-0.665868,-1.091417,-0.423284,-1.208551
10992,99,-3.410771,1.193177,-3.410771,0.608295,-2.874677,-2.081792,-2.053200,-2.080964,-2.039179,...,-1.114959,-1.188775,-1.001034,-1.11151,-1.039125,-1.105188,-0.665868,-1.091417,-0.423284,-1.208551


In [ ]:
predsTest = pd.DataFrame(best_random.predict(test_processed3[final_cols]))
predsTest

,0
0,2
1,2
2,2
3,2
4,2
...,...
10989,2
10990,2
10991,2
10992,2


In [ ]:
predsTest.value_counts()

2    10989
1        5
dtype: int64

In [ ]:
# Takes the generated graph as inpit, along with a d and tolerance value as parameters
def pagerank(graph, d=0.85, tolerance=1e-6):
    
    
    n = len(graph)
    deg = [0] * n
    out_links = [[] for _ in range(n)]

    for i in range(n):
        for j in range(n):
            if graph[i][j]:
                deg[i] += 1
                out_links[j].append(i)

    prob = [[1 / deg[i] if deg[i] else 0 for j in range(n)] for i in range(n)]
    rank = [1 / n] * n

    while True:
        new_rank = [(1 - d) / n + d * sum(rank[j] * prob[j][i] for j in out_links[i])
                    for i in range(n)]
        if sum(abs(new_rank[i] - rank[i]) for i in range(n)) < tolerance:
            return new_rank
        rank = new_rank